In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import logging
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.info("Multi-GPU LightGBM GridSearchCV")

import pm4py
def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    df = pm4py.convert_to_dataframe(log)
    df = df[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
    df = df.sort_values(by=['org:resource', 'time:timestamp'])
    return df

df = import_xes("BPI Challenge 2018.xes")
logging.info(f"✅ Log loaded: {len(df)} events, {df['org:resource'].nunique()} resources.")

le_activity = LabelEncoder()
df['concept:name'] = le_activity.fit_transform(df['concept:name'].astype(str))
num_classes = len(le_activity.classes_)

def create_activity_sequences_encoded(df, prefix_length):
    sequences, next_activities, resources = [], [], []
    for resource, group in df.groupby('org:resource'):
        acts = group['concept:name'].values
        if len(acts) > prefix_length:
            sequences.append(acts[:prefix_length])
            next_activities.append(acts[prefix_length])
            resources.append(resource)
    if not sequences:
        return pd.DataFrame()
    seq_df = pd.DataFrame(sequences, columns=[f"activity_{i+1}" for i in range(prefix_length)])
    seq_df["next_activity"] = next_activities
    seq_df["org:resource"] = resources
    return seq_df

def oversample_proportional_safe(X, y):
    y_series = pd.Series(y)
    counts = y_series.value_counts()
    max_count = counts.max()
    X_resampled, y_resampled = [], []
    for cls in counts.index:
        mask = (y_series == cls)
        X_cls = X[mask]
        y_cls = y_series[mask]
        n_repeat = int(np.ceil(max_count / len(y_cls)))
        X_resampled.append(np.tile(X_cls, (n_repeat, 1)))
        y_resampled.append(np.tile(y_cls, n_repeat))
    return np.vstack(X_resampled), np.hstack(y_resampled)

def run_experiment(df, prefix_length, gpu_id=0):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    logging.info(f"\n🎯 Running prefix={prefix_length} on GPU {gpu_id}")

    start_time = time.time()
    seq_df = create_activity_sequences_encoded(df, prefix_length)
    if seq_df.empty:
        logging.warning(f"⚠️ Skipping prefix_length={prefix_length}: insufficient data.")
        return None

    X = seq_df[[f"activity_{i+1}" for i in range(prefix_length)]].values
    y = seq_df["next_activity"].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True
    )

    X_train, y_train = oversample_proportional_safe(X_train, y_train)

    model = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        boosting_type='gbdt',
        device='gpu',
        random_state=42,
        n_jobs=1,  # one GPU per process
        free_raw_data=False
    )

    param_grid = {
        'n_estimators': [200, 500],
        'max_depth': [10, 20],
        'min_child_samples': [10, 20],
        'min_split_gain': [0.0, 0.1]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=2,
        verbose=1,
        n_jobs=1  # one GPU per process
    )

    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)  

    y_pred = best_model.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
        "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

    runtime = round(time.time() - start_time, 2)
    logging.info(f"🏆 Best params for prefix={prefix_length}: {grid.best_params_}")
    logging.info(f"⏱ Runtime: {runtime}s, GPU {gpu_id}")

    os.makedirs("results/BPIC2018/LightGBM/Baseline encoding", exist_ok=True)
    out_path = f"results/BPIC2018/LightGBM/Baseline encoding/lightgbm_seq_{prefix_length}.json"
    with open(out_path, "w") as f:
        json.dump({
            "sequence_length": prefix_length,
            "best_hyperparameters": grid.best_params_,
            "metrics": metrics,
            "runtime_s": runtime
        }, f, indent=4)

    return {"prefix_length": prefix_length, "metrics": metrics, "runtime_s": runtime}

sequence_lengths = [100, 150, 200, 400, 600, 800, 1000, 1200, 1400, 1500, 2000, 2500]
results = Parallel(n_jobs=4)(
    delayed(run_experiment)(df, seq_len, gpu_id=i % 4)
    for i, seq_len in enumerate(sequence_lengths)
)

logging.info("All GPU GridSearch experiments completed.")


In [ ]:
#SCap
import os
import json
import time
import numpy as np
import pandas as pd
import logging
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.info("Multi-GPU LightGBM GridSearchCV with RA binary features")

import pm4py
def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    df = pm4py.convert_to_dataframe(log)
    df = df[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
    df = df.sort_values(by=['org:resource', 'time:timestamp'])
    return df

df = import_xes("BPI Challenge 2018.xes")
logging.info(f"✅ Log loaded: {len(df)} events, {df['org:resource'].nunique()} resources.")

le_activity = LabelEncoder()
df['concept:name'] = le_activity.fit_transform(df['concept:name'].astype(str))
num_classes = len(le_activity.classes_)

def create_ra_binary(log):
    ra_counts = log.pivot_table(
        index='org:resource', columns='concept:name', aggfunc='size', fill_value=0
    ).reset_index()
    ra_bin = ra_counts.copy()
    ra_bin.iloc[:, 1:] = (ra_bin.iloc[:, 1:] > 0).astype(int)
    return ra_bin, ra_counts.columns[1:].tolist()

ra_binary, ra_activities = create_ra_binary(df)

def create_sequences(df, prefix_length):
    sequences, next_activities, resources = [], [], []
    for resource, g in df.groupby('org:resource'):
        acts = g['concept:name'].values
        if len(acts) > prefix_length:
            sequences.append(acts[:prefix_length])
            next_activities.append(acts[prefix_length])
            resources.append(resource)
    if not sequences:
        return pd.DataFrame()
    seq_df = pd.DataFrame(sequences, columns=[f"activity_{i+1}" for i in range(prefix_length)])
    seq_df['next_activity'] = next_activities
    seq_df['org:resource'] = resources
    return seq_df

def oversample_proportional_safe(X, y):
    y_series = pd.Series(y)
    counts = y_series.value_counts()
    max_count = counts.max()
    X_resampled, y_resampled = [], []
    for cls in counts.index:
        mask = (y_series == cls)
        X_cls = X[mask]
        y_cls = y_series[mask]
        n_repeat = int(np.ceil(max_count / len(y_cls)))
        X_resampled.append(np.tile(X_cls, (n_repeat, 1)))
        y_resampled.append(np.tile(y_cls, n_repeat))
    return np.vstack(X_resampled), np.hstack(y_resampled)

def run_experiment(df, prefix_length, gpu_id=0):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    logging.info(f"\n🎯 Running prefix={prefix_length} on GPU {gpu_id}")

    start_time = time.time()
    seq_df = create_sequences(df, prefix_length)
    if seq_df.empty:
        logging.warning(f"⚠️ Skipping prefix_length={prefix_length}: insufficient data.")
        return None

    ra_filtered = ra_binary[ra_binary['org:resource'].isin(seq_df['org:resource'])].reset_index(drop=True)
    merged_df = pd.concat([seq_df.reset_index(drop=True), ra_filtered.iloc[:, 1:]], axis=1)

    X_cols = [f"activity_{i+1}" for i in range(prefix_length)] + ra_activities
    X = merged_df[X_cols].values
    y = merged_df['next_activity'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    X_train, y_train = oversample_proportional_safe(X_train, y_train)

    model = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        boosting_type='gbdt',
        device='gpu',
        random_state=42,
        n_jobs=1
    )

    param_grid = {
        'n_estimators': [200, 500],
        'max_depth': [10, 20],
        'min_child_samples': [10, 20],
        'min_split_gain': [0.0, 0.1]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=2,
        verbose=1,
        n_jobs=1  # one GPU per process
    )

    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
        "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

    runtime = round(time.time() - start_time, 2)
    logging.info(f"🏆 Best params for prefix={prefix_length}: {grid.best_params_}")
    logging.info(f"⏱ Runtime: {runtime}s, GPU {gpu_id}")

    # Save results
    os.makedirs("results/BPIC2018/LightGBM/SCap", exist_ok=True)
    out_path = f"results/BPIC2018/LightGBM/SCap/lightgbm_ra_seq_{prefix_length}.json"
    with open(out_path, "w") as f:
        json.dump({
            "sequence_length": prefix_length,
            "best_hyperparameters": grid.best_params_,
            "metrics": metrics,
            "runtime_s": runtime
        }, f, indent=4)

    return {"prefix_length": prefix_length, "metrics": metrics, "runtime_s": runtime}

sequence_lengths = [100, 150, 200, 400, 600, 800, 1000, 1200, 1400, 1500, 2000, 2500]
results = Parallel(n_jobs=4)(
    delayed(run_experiment)(df, seq_len, gpu_id=i % 4)
    for i, seq_len in enumerate(sequence_lengths)
)

logging.info("All GPU GridSearch + RA matrix experiments completed.")


In [ ]:
# S2g
import os
import json
import time
import numpy as np
import pandas as pd
import logging
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from joblib import Parallel, delayed
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.info("Multi-GPU LightGBM GridSearchCV with Activity Transitions only")

import pm4py
def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    df = pm4py.convert_to_dataframe(log)
    df = df[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
    df = df.sort_values(by=['org:resource', 'time:timestamp'])
    return df

df = import_xes("BPI Challenge 2018.xes")
logging.info(f"✅ Log loaded: {len(df)} events, {df['org:resource'].nunique()} resources.")

le_activity = LabelEncoder()
df['concept:name'] = le_activity.fit_transform(df['concept:name'].astype(str))
num_classes = len(le_activity.classes_)

def create_sequences(df, prefix_length):
    sequences, next_activities, resources = [], [], []
    for resource, g in df.groupby('org:resource'):
        acts = g['concept:name'].values
        if len(acts) > prefix_length:
            sequences.append(acts[:prefix_length])
            next_activities.append(acts[prefix_length])
            resources.append(resource)
    if not sequences:
        return pd.DataFrame()
    seq_df = pd.DataFrame(sequences, columns=[f"activity_{i+1}" for i in range(prefix_length)])
    seq_df['next_activity'] = next_activities
    seq_df['org:resource'] = resources
    return seq_df

def create_transition_features(seq_df):
    activity_cols = [c for c in seq_df.columns if c.startswith('activity_')]
    unique_activities = sorted(set(seq_df[activity_cols].values.flatten()))
    all_transitions = [(a, b) for a in unique_activities for b in unique_activities]
    transition_features = []
    for _, row in seq_df.iterrows():
        transitions = defaultdict(int)
        acts = row[activity_cols].tolist()
        for i in range(len(acts)-1):
            transitions[(acts[i], acts[i+1])] += 1
        row_counts = {f"{a}->{b}": transitions.get((a,b),0) for (a,b) in all_transitions}
        transition_features.append(row_counts)
    return pd.concat([seq_df.reset_index(drop=True), pd.DataFrame(transition_features)], axis=1)

def oversample_proportional_safe(X, y):
    y_series = pd.Series(y)
    counts = y_series.value_counts()
    max_count = counts.max()
    X_resampled, y_resampled = [], []
    for cls in counts.index:
        mask = (y_series == cls)
        X_cls = X[mask]
        y_cls = y_series[mask]
        n_repeat = int(np.ceil(max_count / len(y_cls)))
        X_resampled.append(np.tile(X_cls, (n_repeat, 1)))
        y_resampled.append(np.tile(y_cls, n_repeat))
    return np.vstack(X_resampled), np.hstack(y_resampled)

def run_experiment(df, prefix_length, gpu_id=0):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    logging.info(f"\n🎯 Running prefix={prefix_length} on GPU {gpu_id}")
    start_time = time.time()

    seq_df = create_sequences(df, prefix_length)
    if seq_df.empty:
        logging.warning(f"⚠️ Skipping prefix_length={prefix_length}: insufficient data.")
        return None

    # Add transition features
    seq_df = create_transition_features(seq_df)

    for col in seq_df.columns:
        if seq_df[col].dtype == 'object':
            seq_df[col] = LabelEncoder().fit_transform(seq_df[col].astype(str))

    # Split features/labels
    X_cols = [c for c in seq_df.columns if c not in ['next_activity','org:resource']]
    X = seq_df[X_cols].values
    y = seq_df['next_activity'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
    X_train, y_train = oversample_proportional_safe(X_train, y_train)

    model = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        boosting_type='gbdt',
        device='gpu',
        random_state=42,
        n_jobs=1
    )

    param_grid = {
        'n_estimators': [200, 500],
        'max_depth': [10, 20],
        'min_child_samples': [10, 20],
        'min_split_gain': [0.0, 0.1]
    }

    grid = GridSearchCV(model, param_grid, scoring='accuracy', cv=2, verbose=1, n_jobs=1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
        "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

    runtime = round(time.time() - start_time, 2)
    logging.info(f"🏆 Best params for prefix={prefix_length}: {grid.best_params_}")
    logging.info(f"⏱ Runtime: {runtime}s, GPU {gpu_id}")

    # Save results
    os.makedirs("results/BPIC2018/LightGBM/S2g", exist_ok=True)
    out_path = f"results/BPIC2018/LightGBM/S2g/lightgbm_trans_seq_{prefix_length}.json"
    with open(out_path, "w") as f:
        json.dump({
            "sequence_length": prefix_length,
            "best_hyperparameters": grid.best_params_,
            "metrics": metrics,
            "runtime_s": runtime
        }, f, indent=4)

    return {"prefix_length": prefix_length, "metrics": metrics, "runtime_s": runtime}

sequence_lengths = [100, 150, 200, 400, 600, 800, 1000, 1200, 1400, 1500, 2000, 2500]
results = Parallel(n_jobs=4)(
    delayed(run_experiment)(df, seq_len, gpu_id=i % 4)
    for i, seq_len in enumerate(sequence_lengths)
)

logging.info("All GPU GridSearch + Transition-only experiments completed.")


In [ ]:
#S2gR
import os
import json
import time
import numpy as np
import pandas as pd
import logging
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from joblib import Parallel, delayed
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.info("Multi-GPU LightGBM GridSearchCV with Transition + Repeat Features")

import pm4py
def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    df = pm4py.convert_to_dataframe(log)
    df = df[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
    df = df.sort_values(by=['org:resource', 'time:timestamp']).reset_index(drop=True)
    return df

df = import_xes("BPI_Challenge_2013_incidents.xes")
logging.info(f"✅ Log loaded: {len(df)} events, {df['org:resource'].nunique()} resources.")

le_activity = LabelEncoder()
df['concept:name'] = le_activity.fit_transform(df['concept:name'].astype(str))
num_classes = len(le_activity.classes_)

def create_activity_sequences_encoded(df, prefix_length):
    sequences, next_activities, resources = [], [], []
    for resource, group in df.groupby('org:resource'):
        acts = group['concept:name'].values
        if len(acts) > prefix_length:
            sequences.append(acts[:prefix_length])
            next_activities.append(acts[prefix_length])
            resources.append(resource)
    if not sequences:
        return pd.DataFrame()
    seq_df = pd.DataFrame(sequences, columns=[f"activity_{i+1}" for i in range(prefix_length)])
    seq_df["next_activity"] = next_activities
    seq_df["org:resource"] = resources
    return seq_df

def create_transition_and_repeat_features(seq_df):
    activity_cols = [c for c in seq_df.columns if c.startswith('activity_')]
    unique_activities = sorted(set(seq_df[activity_cols].values.flatten()))
    all_transitions = [(a, b) for a in unique_activities for b in unique_activities]

    transition_features = []
    repeat_features = []

    for _, row in seq_df.iterrows():
        acts = row[activity_cols].tolist()
        
        # Transition counts
        transitions = defaultdict(int)
        for i in range(len(acts)-1):
            transitions[(acts[i], acts[i+1])] += 1
        row_transitions = {f"{a}->{b}": transitions.get((a,b), 0) for (a,b) in all_transitions}
        transition_features.append(row_transitions)
        
        # Repeat pattern features
        run_lengths = []
        current_run = 1
        for i in range(1, len(acts)):
            if acts[i] == acts[i-1]:
                current_run += 1
            else:
                run_lengths.append(current_run)
                current_run = 1
        run_lengths.append(current_run)
        repeat_features.append({
            "avg_run_length": np.mean(run_lengths),
            "num_runs": len(run_lengths)
        })

    transitions_df = pd.DataFrame(transition_features)
    repeat_df = pd.DataFrame(repeat_features)
    return pd.concat([seq_df.reset_index(drop=True), transitions_df, repeat_df], axis=1)

def oversample_proportional_safe(X, y):
    y_series = pd.Series(y)
    counts = y_series.value_counts()
    max_count = counts.max()
    X_resampled, y_resampled = [], []
    for cls in counts.index:
        mask = (y_series == cls)
        X_cls = X[mask]
        y_cls = y_series[mask]
        n_repeat = int(np.ceil(max_count / len(y_cls)))
        X_resampled.append(np.tile(X_cls, (n_repeat, 1)))
        y_resampled.append(np.tile(y_cls, n_repeat))
    return np.vstack(X_resampled), np.hstack(y_resampled)


def run_experiment(df, prefix_length, gpu_id=0):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    logging.info(f"\n🎯 Running prefix={prefix_length} on GPU {gpu_id}")

    start_time = time.time()
    seq_df = create_activity_sequences_encoded(df, prefix_length)
    if seq_df.empty:
        logging.warning(f"⚠️ Skipping prefix_length={prefix_length}: insufficient data.")
        return None

    # Add transition + repeat features
    seq_df = create_transition_and_repeat_features(seq_df)

    feature_cols = [c for c in seq_df.columns if c not in ['next_activity','org:resource']]
    X = seq_df[feature_cols].values
    y = seq_df["next_activity"].values

    # Split train/test BEFORE oversampling
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True
    )

    # Oversample training set only
    X_train, y_train = oversample_proportional_safe(X_train, y_train)

    # LightGBM model
    model = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        boosting_type='gbdt',
        device='gpu',
        random_state=42,
        n_jobs=1,  # one GPU per process
        free_raw_data=False
    )

    # Parameter grid
    param_grid = {
        'n_estimators': [200, 500],
        'max_depth': [10, 20],
        'min_child_samples': [10, 20],
        'min_split_gain': [0.0, 0.1]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=2,
        verbose=1,
        n_jobs=1  # one GPU per process
    )

    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
        "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

    runtime = round(time.time() - start_time, 2)
    logging.info(f"🏆 Best params for prefix={prefix_length}: {grid.best_params_}")
    logging.info(f"⏱ Runtime: {runtime}s, GPU {gpu_id}")

    os.makedirs("results/BPIC2013/LightGBM/S2gR", exist_ok=True)
    out_path = f"results/BPIC2013/LightGBM/S2gR/lightgbm_seq_{prefix_length}.json"
    with open(out_path, "w") as f:
        json.dump({
            "sequence_length": prefix_length,
            "best_hyperparameters": grid.best_params_,
            "metrics": metrics,
            "runtime_s": runtime
        }, f, indent=4)

    return {"prefix_length": prefix_length, "metrics": metrics, "runtime_s": runtime}

sequence_lengths = [10,20,30,40,50,75,100,125,150]
results = Parallel(n_jobs=4)(
    delayed(run_experiment)(df, seq_len, gpu_id=i % 4)
    for i, seq_len in enumerate(sequence_lengths)
)

logging.info("All GPU GridSearch experiments completed.")
